In [2]:
from api_helper import ShoonyaApiPy
import logging
import time
import pandas as pd
import json
import numpy as np
import cred
from datetime import datetime, timedelta
import time
from notify_run import Notify
notify = Notify()
import pyotp
token = cred.token
totp = pyotp.TOTP(token).now()
#enable dbug to see request and responses
logging.basicConfig(level=logging.DEBUG)
#start of our program
api = ShoonyaApiPy()
#credentials
user = cred.user
pwd = cred.pwd
factor2 = totp
vc = cred.vc
app_key = cred.apikey
imei = cred.imei
logins = api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, 
    api_secret=app_key,imei = imei)
print(logins)

DEBUG:NorenRestApiPy.NorenApi:https://api.shoonya.com/NorenWClientTP//QuickAuth
DEBUG:NorenRestApiPy.NorenApi:Req:jData={"source": "API", "apkversion": "1.0.0", "uid": "FA91335", "pwd": "dbc20194ef429540e62dd01f626f90c03015e9857d44737bebe3231633645cc4", "factor2": "639017", "vc": "FA91335_U", "appkey": "d33bc9b7243f81379ffdef5b690c5b24c8dc14d14ac97bec458d47bb2a139e54", "imei": "FA91335_U"}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.shoonya.com:443
DEBUG:urllib3.connectionpool:https://api.shoonya.com:443 "POST /NorenWClientTP//QuickAuth HTTP/1.1" 200 2212
DEBUG:NorenRestApiPy.NorenApi:Reply:{"request_time":"09:48:38 22-05-2023","actid":"FA91335","access_type":["WEB","TT","MOB","API"],"uname":"VIKAS  KUMAR","prarr":[{"prd":"C","s_prdt_ali":"CNC","exch":["NSE","BSE","NIPO","BSTAR"]},{"prd":"M","s_prdt_ali":"NRML","exch":["NFO","CDS","BCD"]},{"prd":"I","s_prdt_ali":"MIS","exch":["NSE","BSE","NFO","CDS","BCD"]},{"prd":"H","s_prdt_ali":"CO","exch":["NSE","NFO","CDS",

{'request_time': '09:48:38 22-05-2023', 'actid': 'FA91335', 'access_type': ['WEB', 'TT', 'MOB', 'API'], 'uname': 'VIKAS  KUMAR', 'prarr': [{'prd': 'C', 's_prdt_ali': 'CNC', 'exch': ['NSE', 'BSE', 'NIPO', 'BSTAR']}, {'prd': 'M', 's_prdt_ali': 'NRML', 'exch': ['NFO', 'CDS', 'BCD']}, {'prd': 'I', 's_prdt_ali': 'MIS', 'exch': ['NSE', 'BSE', 'NFO', 'CDS', 'BCD']}, {'prd': 'H', 's_prdt_ali': 'CO', 'exch': ['NSE', 'NFO', 'CDS', 'BSE']}, {'prd': 'B', 's_prdt_ali': 'BO', 'exch': ['NSE', 'NFO', 'CDS', 'BSE']}], 'stat': 'Ok', 'susertoken': '74472483877debcf30ec9e0609472bbd0b5f74d76efdf1c04073e8eff466cc86', 'email': 'vikaslosal10@gmail.com', 'uid': 'FA91335', 'brnchid': 'HO', 'totp_set': '1', 'orarr': ['MKT', 'LMT', 'SL-LMT', 'SL-MKT'], 'exarr': ['NSE', 'NIPO', 'NFO', 'CDS', 'BSE', 'BCD', 'BSTAR'], 'values': ['3', '2', '1'], 'mws': {'3': [{'exch': 'NFO', 'token': '43741', 'tsym': 'MIDCPNIFTY21JUN23P8350', 'weekly': 'W3', 'dname': 'MIDCPNIFTY 21JUN23 8350 PE ', 'instname': 'OPTIDX', 'pp': '2', 'ls'

In [ ]:
strike_array = ['C19000','C19500','P17000','P16500','P17500']
Q = 50
q_exit= 50000

In [ ]:
# api.single_order_history('23051700032952')[0]

In [ ]:
def strategy(buy,Q,q_exit,buy_no):
    for i in range(1):                    # for buy orders on 3 ticks
        nikal = False
        change = False
        phek = False
        phek2=False
        upr_gya = False

        while(1):
    #         break
            buy_status = api.single_order_history(buy_no[i])[0]['status']

            bp1 = float(api.get_quotes('NFO',st)['bp1'])
            if(bp1<buy):
                nikal = True
                break
            elif(buy_status=='COMPLETE'):
                nikal = False
                break
                
            if((i==0)&(bp1>=buy+0.1)):
                upr_gya = True
                break
                #till now I know ki buy at buy or due to dec in bp1
    #     print('out')
    
        if(upr_gya==True):
            api.cancel_order(buy_no[0])
            api.cancel_order(buy_no[1])
            break
        
        if(nikal==True):
                            #bought due to dec in bp1
            order_no = api.place_order(buy_or_sell='S', product_type='I',exchange='NFO', tradingsymbol=st, 
            quantity=Q, discloseqty=0,price_type='LMT', price=buy, 
            trigger_price=None,
             retention='DAY')['norenordno']             # sell at bhav to bhav

            while(1):
                x = api.get_quotes('NFO',st)
                bp1 = float(x['bp1'])
                bq1 = int(x['bq1'])
                status = api.single_order_history(order_no)[0]['status']
                if(status=='COMPLETE'):
                    phek = False
                    break

                elif((bq1<q_exit) | (bp1<buy-0.05)):
                    phek = True
                    break

            if(phek==True):
                api.cancel_order(order_no)

                api.place_order(buy_or_sell='S', product_type='I',exchange='NFO', tradingsymbol=st,  
                quantity=Q, discloseqty=0,price_type='LMT', price=0, 
                trigger_price=None,
                 retention='DAY')


                buy = buy-0.05

            else:
                buy = buy -0.05


        #shi price pe buy ho gya, ab next


        else:
            order_no = api.place_order(buy_or_sell='S', product_type='I',exchange='NFO', tradingsymbol=st, 
            quantity=Q, discloseqty=0,price_type='LMT', price=buy+0.05, 
            trigger_price=None,
             retention='DAY')['norenordno']     

            while(1):
                x = api.get_quotes('NFO',st)
                bp1 = float(x['bp1'])
                bq1 = int(x['bq1'])

                status = api.single_order_history(order_no)[0]['status']
                if(status=='COMPLETE'):
                    change = False
                    break
                elif((bp1<buy) | (bq1<q_exit)):    #change your sell order 
                    change = True
                    break
    #         print('here')
            if(change==True):    #change order
                api.cancel_order(order_no)
                order_no = api.place_order(buy_or_sell='S', product_type='I',exchange='NFO', tradingsymbol=st, 
                quantity=Q, discloseqty=0,price_type='LMT', price=buy, 
                trigger_price=None,
                 retention='DAY')['norenordno']
                time.sleep(2)

                while(1):
                    status = api.single_order_history(order_no)[0]['status']
                    x = api.get_quotes('NFO',st)
                    bp1 = float(x['bp1'])
                    bq1 = int(x['bq1'])

                    if(status=='COMPLETE'):
                        phek2 = False
                        break

                    elif((bq1<q_exit) | (bp1<buy-0.05)):
                        phek2 = True
                        break

                if(phek2==True):
                    api.cancel_order(order_no)

                    api.place_order(buy_or_sell='S', product_type='I',exchange='NFO', tradingsymbol=st, 
                    quantity=Q, discloseqty=0,price_type='LMT', price=0, 
                    trigger_price=None,
                     retention='DAY')

                    buy = buy-0.05

                else:
                    buy = buy -0.05
            else:
                buy = buy-0.05

            
            


In [ ]:
df = []
for i in range(5):
    df.append(pd.DataFrame(columns = ['bq1','bp1','sq1','v']))

In [ ]:
buy = 0
st = 0
sq=sp=bq=bp=v=0
feed_opened = False
signal = False

In [ ]:

def event_handler_feed_update(m):
    #print(m)
    global feed_opened,df,a,sq,sp,bq,bp,v,st,buy, signal
    
    ind = np.where(a==m['tk'])[0][0]
    
    if('sq1'in m):
        sq = float(m['sq1'])
    else:
        sq = df[ind]['sq1'][df[ind]['sq1'].count()-1]
        
#     if('sp1'in m):
#         sp = float(m['sp1'])
#     else:
#         sp = df[ind]['sp1'][df[ind]['sp1'].count()-1]
    if('bq1'in m):
        bq = float(m['bq1'])
    else:
        bq = df[ind]['bq1'][df[ind]['bq1'].count()-1]
    if('bp1'in m):
        bp = float(m['bp1'])
    else:
        bp = df[ind]['bp1'][df[ind]['bp1'].count()-1]
    if('v'in m):
        v = float(m['v'])
    else:
        v = df[ind]['v'][df[ind]['v'].count()-1]
    
    df[ind].loc[df[ind]['v'].count()] = [bq,bp,sq,v]
    
    if((signal==False)&(df[ind]['v'].count()>15)):
        
        if((bq/sq > 2) &(df[ind]['v'][df[ind]['v'].count()-1]/df[ind]['v'][df[ind]['v'].count()-11]>1.01)):
            signal = True
            st = ind
            buy = bp
    
    
    
        
def event_handler_order_update(inmessage):
    print(inmessage)                                      #     if(('norennordno' in inmessage)&('status'in inmessage)):  #         orderJson[inmessage['norenordno']] = {'status': inmessage['status']}
        
def open_callback():
    
    global feed_opened
    feed_opened = True



In [ ]:
api.start_websocket( order_update_callback=event_handler_order_update,
 subscribe_callback=event_handler_feed_update, 
socket_open_callback=open_callback)
while(feed_opened==False):
    pass


In [ ]:
a =np.zeros(5)


for i in range(5):
    
    b = api.searchscrip('NFO','NIFTY18MAY23'+strike_array[i])
    a[i] = b['values'][0]['token']
    


In [ ]:
b = []
for i in range(5):
    b.append('NFO|'+str(int(a[i])))
a = a.astype(int)
a = a.astype(str)

In [ ]:
api.subscribe(b)

In [ ]:
while(1):
    
    
    curr = datetime.now()
    new = curr+timedelta(minutes = 2)
    while(1):
        while(curr.minute != new.minute):
            if(signal==True):
                break
            curr = datetime.now()
        if(signal==True):
            api.unsubscribe(b)
            api.close_websocket()
            feed_opened = False
            break
        api.unsubscribe(b)
        api.close_websocket()

        feed_opened = False

        api.start_websocket( order_update_callback=event_handler_order_update, subscribe_callback=event_handler_feed_update, socket_open_callback=open_callback)
        while(feed_opened==False):
            pass
        api.subscribe(b)

        new = curr + timedelta(minutes = 2)


    #now i got signal to buy at 'buy', to strike with index st in strike_array


    #nikal  = False   #will true when biught due to dec. in bp1
    #change  = False   #when I have to change my sell order either due to dec in bp1 or in bq1
    #phek   = False    #phek de at loss
    #phek2 = False    #vo phek jo buy pr hi buy hua but ab :(
    st = 'NIFTY18MAY23'+strike_array[st]     # this will be trading symbol which we will be using to buy sell


    #now buy 
    buy_no = ['','']
    buy_no[0] = api.place_order(buy_or_sell='B', product_type='I',exchange='NFO', tradingsymbol=st, 
    quantity=Q, discloseqty=0,price_type='LMT', price=buy, 
    trigger_price=None,
     retention='DAY')['norenordno']
    buy_no[1] = api.place_order(buy_or_sell='B', product_type='I',exchange='NFO', tradingsymbol=st, 
    quantity=Q, discloseqty=0,price_type='LMT', price=buy-0.05, 
    trigger_price=None,
     retention='DAY')['norenordno']


    #its time to call strategy
    strategy(buy,Q,q_exit, buy_no)


    #setting again for next trades and while loop
    signal = False
    api.start_websocket( order_update_callback=event_handler_order_update,
     subscribe_callback=event_handler_feed_update, 
    socket_open_callback=open_callback)
    while(feed_opened==False):
        pass
    api.subscribe(b)


In [ ]:
signal

In [ ]:
api.get_positions()

In [ ]:
b=2

In [ ]:
position[0]['netqty']

In [ ]:
api.get_quotes('NFO','FINNIFTY16MAY23P19200')

In [ ]:
api.place_order(buy_or_sell='B', product_type='M',exchange='NFO', tradingsymbol=st, 
                quantity=Q, discloseqty=0,price_type='LMT', price=0.55, 
                trigger_price=None,
                 retention='DAY')

In [ ]:
quant = int(api.get_positions()['netqty'])

In [ ]:
(api.get_positions()) is None

In [ ]:
api.get_quotes('NFO',st)

In [ ]:
api.get_daily_price_series('NFO',st)

In [ ]:
        api.place_order(buy_or_sell='S', product_type='M',exchange='NFO', tradingsymbol=st, 
        quantity=Q, discloseqty=0,price_type='LMT', price=buy+0.05, 
        trigger_price=None,
         retention='DAY')

In [ ]:
api.single_order_history('23015300006823')

In [ ]:
order_no =['','',''] 

In [ ]:
order_no = []

In [ ]:
order_no.append(api.sell(dljfkjvkfmvlfdv))
order_no.append(knfd)


In [ ]:
def(sell, len(order_no):
    for order in order_no:
        

In [ ]:
def sell

In [ ]:

# while(b<3):                    # for buy orders on 3 ticks
#     b= b+1
#     nikal = False
#     change = False
#     phek = False


#     while(1):
# #         break
#         position = api.get_positions()
#         if position is None:
#             quant = 0
#         else:
            
#             quant = int(position['netqty'])
#         bp1 = float(api.get_quotes('NFO',st)['bp1'])
#         if(bp1<buy):
#             nikal = True
#             break
#         elif(quant==Q):
#             nikal = False
#             break
#             #till now I know ki buy at buy or due to dec in bp1
# #     print('out')
#     if(nikal==True):
#                         #bought due to dec in bp1
#         order_no = api.place_order(buy_or_sell='S', product_type='M',exchange='NFO', tradingsymbol=st, 
#         quantity=Q, discloseqty=0,price_type='LMT', price=buy, 
#         trigger_price=None,
#          retention='DAY')['norenordno']             # sell at bhav to bhav
        
#         while(1):
            
#             status = api.single_order_history(order_no)
#             if(status=='COMPLETE'):
#                 phek = False
#                 break
#             x = api.get_quotes('NFO',st)
#             bp1 = float(x['bp1'])
#             bq1 = int(x['bq1'])
#             if((bq1<q_exit) | (bp1<buy-0.05)):
#                 phek = True
#                 break

#         if(phek==True):
#             api.cancel_order(order_no)
            
#             api.place_order(buy_or_sell='S', product_type='M',exchange='NFO', tradingsymbol=st,  
#             quantity=Q, discloseqty=0,price_type='LMT', price=0, 
#             trigger_price=None,
#              retention='DAY')
            
            
#             buy = buy-0.05

#         else:
#             buy = buy -0.05

    
#     #shi price pe buy ho gya, ab next


#     else:
#         order_no = api.place_order(buy_or_sell='S', product_type='M',exchange='NFO', tradingsymbol=st, 
#         quantity=Q, discloseqty=0,price_type='LMT', price=buy+0.05, 
#         trigger_price=None,
#          retention='DAY')['norenordno']     
        
#         while(1):
#             x = api.get_quotes('NFO',st)
#             bp1 = float(x['bp1'])
#             bq1 = int(x['bq1'])

#             status = api.single_order_history(order_no)
#             if(status=='COMPLETE'):
#                 change = False
#                 break
#             if((bp1<buy) | (bq1<q_exit)):    #change your sell order 
#                 change = True
#                 break
# #         print('here')
#         if(change==True):    #change order
#             api.cancel_order(order_no)
#             order_no = api.place_order(buy_or_sell='S', product_type='M',exchange='NFO', tradingsymbol=st, 
#             quantity=Q, discloseqty=0,price_type='LMT', price=buy, 
#             trigger_price=None,
#              retention='DAY')['norenordno']
            
#             while(1):
#                 status = api.single_order_history(order_no)
#                 if(status=='COMPLETE'):
#                     phek2 = False
#                     break
#                 x = api.get_quotes('NFO',st)
#                 bp1 = float(x['bp1'])
#                 bq1 = int(x['bq1'])
#                 if((bq1<q_exit) | (bp1<buy-0.05)):
#                     phek2 = True
#                     break

#             if(phek2==True):
#                 api.cancel_order(order_no)
            
#                 api.place_order(buy_or_sell='S', product_type='M',exchange='NFO', tradingsymbol=st, 
#                 quantity=Q, discloseqty=0,price_type='LMT', price=0, 
#                 trigger_price=None,
#                  retention='DAY')
            
#                 buy = buy-0.05

#             else:
#                 buy = buy -0.05
#         else:
#             buy = buy-0.05
